In [1]:
import folium
import geopandas as gp
import pandas as pd

*read all the needed datasets*

In [2]:
gp_subwayline = gp.read_file('../Data/Cleaned/subwaylines/subwayLines.shp')
gp_subwaystn = gp.read_file('../Data/Cleaned/subwaystops/subwayStations.shp')

In [3]:
df_neighbourhoods = gp.read_file('../Data/Cleaned/neighbourhoods.geojson')

In [4]:
df_neighbourhoods.head()

,neighbourhood,neighbourhood_group,geometry
0,York University Heights,None,"MULTIPOLYGON (((-79.50529 43.75987, -79.50488 ..."
1,The Beaches,None,"MULTIPOLYGON (((-79.31485 43.66674, -79.31356 ..."
2,Scarborough Village,None,"MULTIPOLYGON (((-79.21397 43.72676, -79.21400 ..."
3,South Parkdale,None,"MULTIPOLYGON (((-79.42777 43.62979, -79.42948 ..."
4,South Riverdale,None,"MULTIPOLYGON (((-79.32868 43.64745, -79.32867 ..."


In [103]:
gp_subwaystn.head()

,STATION,LINE,PLATFORM_L,AVG_PASSEN,LINE2,PLATFORM_1,SUBWAY_TRA,ADDRESS,Opened,geometry,geom_str,LONGITUDE,LATITUDE
0,Kipling,Bloor-Danforth,1,53640,None,None,False,5247 Dundas St. West,1980,POINT (-79.53583 43.63734),POINT (-79.5358325279170089 43.6373449292934481),-79.5358325279170089,43.6373449292934481
1,Islington,Bloor-Danforth,1,43090,None,None,False,3286 Bloor St. West,1968,POINT (-79.52461 43.64537),POINT (-79.5246147296282686 43.6453745598367675),-79.5246147296282686,43.6453745598367675
2,Royal York,Bloor-Danforth,2,19440,None,None,False,3012 Bloor St. West,1968,POINT (-79.51133 43.64825),POINT (-79.5113277455817951 43.6482514461941875),-79.5113277455817951,43.6482514461941875
3,Old Mill,Bloor-Danforth,2,5780,None,None,False,2672 Bloor St. West,1968,POINT (-79.49510 43.65010),POINT (-79.4950977657188957 43.6501020255896393),-79.4950977657188957,43.6501020255896393
4,Jane,Bloor-Danforth,2,16730,None,None,False,2440 Bloor St. West,1968,POINT (-79.48445 43.64978),POINT (-79.4844545118749295 43.6497766253667407),-79.4844545118749295,43.6497766253667407


In [101]:
from shapely import wkt

for index in range(0,len(gp_subwaystn)):
    gp_subwaystn['geom_str'][index] = wkt.dumps(gp_subwaystn['geometry'][index])

<ipython-input-101-e18fc89d2a71>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gp_subwaystn['geom_str'][index] = wkt.dumps(gp_subwaystn['geometry'][index])


In [96]:
len(gp_subwaystn)

65

In [51]:
def longlat(row):
    row = row.split('(')[1].split(')')[0]
    long , lat = row.split(' ')
    return pd.Series([long,lat])

In [102]:
gp_subwaystn[['LONGITUDE','LATITUDE']] = gp_subwaystn.geom_str.apply(longlat)

In [6]:
gp_subwayline.head()

,LINE,SHAPE_Leng,geometry
0,Bloor-Danforth,26377.465093,"LINESTRING (618101.613 4832636.300, 618185.830..."
1,Sheppard,5431.438569,"LINESTRING (627911.441 4846611.990, 629033.015..."
2,TransferTunnel,351.921082,"LINESTRING (628562.925 4836492.718, 628675.035..."
3,Yonge-University-Spadina,30085.504598,"MULTILINESTRING ((630849.699 4834078.548, 6308..."


*assign different colors to the transit lines*

In [7]:
gp_subwayline['colors'] = ['red', 'white','blue','black']

for dataset in [gp_subwayline, gp_subwaystn]:
    dataset['geometry'] = dataset['geometry'].to_crs({'init': 'EPSG:4269'})
    


/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [8]:
gp_subwayline

,LINE,SHAPE_Leng,geometry,colors
0,Bloor-Danforth,26377.465093,"LINESTRING (-79.53583 43.63734, -79.53477 43.6...",red
1,Sheppard,5431.438569,"LINESTRING (-79.41094 43.76151, -79.39692 43.7...",white
2,TransferTunnel,351.921082,"LINESTRING (-79.40527 43.67032, -79.40396 43.6...",blue
3,Yonge-University-Spadina,30085.504598,"MULTILINESTRING ((-79.37750 43.64819, -79.3776...",black


*plotting maps*

In [113]:
toronto_coord = [43.702270, -79.366074]

map_ = folium.Map(location=toronto_coord, zoom_start=10,prefer_canvas=True)

In [16]:
gp_subwayline.crs = 'epsg:4326'

In [120]:
folium.Choropleth(
    gp_subwayline[gp_subwayline.geometry.length>0.001],
    line_weight=3,
    line_color='blue'
).add_to(map_) 

<ipython-input-120-05449527ae1b>:2: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gp_subwayline[gp_subwayline.geometry.length>0.001],


In [107]:
substn = gp_subwaystn[['LATITUDE','LONGITUDE']]
substn_list = substn.values.tolist()

In [90]:
len(substn_list)

65

In [122]:
for point in range(0, len(substn_list)):
    folium.Circle(substn_list[point], popup=gp_subwaystn['STATION'][point], radius =20,
                 color='white').add_to(map_)

In [123]:
map_